In [1]:
import pathlib

In [2]:
import joblib
import lightgbm as lgb
import pandas as pd

In [3]:
root_dir_path = pathlib.Path("..")
data_dir_path = root_dir_path / "data"
processed_dir_path = data_dir_path / "processed"
train_path = processed_dir_path / "train.parquet"
models_dir_path = root_dir_path / "models"
model_path = models_dir_path / "model.joblib"

In [4]:
train = pd.read_parquet(train_path)

In [5]:
data = train.select_dtypes("number")

In [6]:
data.describe(include="all")

,demand,wm_yr_wk,snap_CA,snap_TX,snap_WI,sell_price
count,5.832737e+07,5.832737e+07,5.832737e+07,5.832737e+07,5.832737e+07,4.602796e+07
mean,1.126322e+00,1.133919e+04,3.293257e-01,3.293257e-01,3.293257e-01,4.408089e+00
std,3.873108e+00,1.503742e+02,4.699684e-01,4.699684e-01,4.699684e-01,3.403657e+00
min,0.000000e+00,1.110100e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e-02
25%,0.000000e+00,1.121700e+04,0.000000e+00,0.000000e+00,0.000000e+00,2.180000e+00
50%,0.000000e+00,1.133300e+04,0.000000e+00,0.000000e+00,0.000000e+00,3.470000e+00
75%,1.000000e+00,1.144800e+04,1.000000e+00,1.000000e+00,1.000000e+00,5.840000e+00
max,7.630000e+02,1.161300e+04,1.000000e+00,1.000000e+00,1.000000e+00,1.073200e+02


In [7]:
X = data
y = data.pop("demand")

In [8]:
model = lgb.LGBMRegressor(random_state=0)

In [9]:
model.fit(X, y)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [10]:
joblib.dump(model, model_path)

['..\\models\\model.joblib']